

# wGMM–UAPCA Evaluation Demonstration

This notebook presents the **quantitative evaluation** of the wGMM–UAPCA method, as described by Klötzl et al. in [*Uncertainty-Aware PCA for Arbitrarily Distributed Data Modeled by Gaussian Mixture Models*](https://arxiv.org/abs/2508.13990).

We focus on **comparing the quality of 2D projections** generated by wGMM–UAPCA against baseline methods such as standard PCA and UAPCA. The evaluation employs the following metrics:

- **Kullback–Leibler (KL) divergence**
- **Sliced Wasserstein distance**

This notebook evaluates how well wGMM–UAPCA preserves label-specific structures in low-dimensional representations.

*Note:* Minor numerical differences may occur across systems due to implementation details of `scipy.stats.gaussian_kde` and stochastic sampling in the Sliced Wasserstein distance computation. These discrepancies may lead to slight variations between the results obtained here and those reported in the paper above, but they do not affect the qualitative conclusions of the evaluation.

## 1. Imports and Setup

Load the main `wgmm_uapca` library and packages needed and set random seed of NumPy for reproducibility.

In [1]:
import wgmm_uapca as wgmm

import numpy as np
import pandas as pd
import os
import ot

np.random.seed(0)

## 2. Helper Functions

Define utility functions for converting 2D density grids to samples, and for computing **KL divergence** and **sliced Wasserstein distance** between projected distributions.

In [2]:
def _density_to_samples(density: np.ndarray, n_samples: int = 1000) -> np.ndarray:
    """
    Convert a 2D density grid into samples using weighted sampling.

    Parameters
    ----------
    density : np.ndarray
        2D density array.
    n_samples : int, default=1000
        Number of samples to draw.

    Returns
    -------
    samples : np.ndarray
        Array of sampled points of shape (n_samples, 2).
    """
    density = density / np.sum(density)

    h, w = density.shape
    X, Y = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    coords = np.stack([X.ravel(), Y.ravel()], axis=1)

    indices = np.random.choice(len(coords), size=n_samples, p=density.ravel())

    return coords[indices]


def compute_sliced_wasserstein_distance(density1: np.ndarray, density2: np.ndarray, n_samples: int = 5000, n_projections: int = 50) -> float:
    """
    Compute the sliced Wasserstein distance between two 2D densities.

    Parameters
    ----------
    density1 : np.ndarray
        First 2D density array.
    density2 : np.ndarray
        Second 2D density array.
    n_samples : int, default=1000
        Number of samples drawn from each density.
    n_projections : int, default=50
        Number of random 1D projections.

    Returns
    -------
    swd : float
        Scalar sliced Wasserstein distance.
    """
    X_a = _density_to_samples(density1, n_samples)
    X_b = _density_to_samples(density2, n_samples)

    return ot.sliced.sliced_wasserstein_distance(X_a, X_b, n_projections=n_projections, seed=0)


def compute_kl_divergence(ref: np.ndarray, comp: np.ndarray, grid: tuple[np.ndarray, np.ndarray]) -> float:
    """
    Compute the KL divergence D_KL(ref || comp) between two 2D densities on a grid.

    Parameters
    ----------
    ref : np.ndarray
        Reference 2D density array.
    comp : np.ndarray
        Comparison 2D density array.
    grid : tuple[np.ndarray, np.ndarray]
        Meshgrid arrays (xx, yy) defining the grid.

    Returns
    -------
    kl : float
        Scalar KL divergence value.
    """
    xx, yy = grid
    eps = 1e-10

    p = ref / np.sum(ref)
    q = comp / np.sum(comp)

    mask = q > 0

    dx = xx[0, 1] - xx[0, 0]
    dy = yy[1, 0] - yy[0, 0]

    return np.sum(p[mask] * (np.log(p[mask] + eps) - np.log(q[mask] + eps))) * dx * dy

## 3. Evaluation

In this step, we load multiple benchmark datasets provided by [Espadoto et al. (2021)](https://doi.org/10.1109/TVCG.2019.2944182) and prepare them for evaluation. For each dataset, we:

1. **Fit Gaussian Mixture Models (GMMs)** for each label to model label-specific distributions.
2. **Compute label weights** based on the number of samples per label.
3. **Calculate 2D projections** using WGMM-UAPCA, UAPCA, and standard PCA.
4. **Generate density grids** for each method to enable distribution comparison.
5. **Visualize and save projections** along with density contours for qualitative assessment.
6. **Compute quantitative measures** (KL divergence and sliced Wasserstein distance) for each label and method comparison.

All computed measures are saved as a CSV file into `results/measures.csv`.

In [ ]:
# Load datasets or download them if not downloaded yet
dataset_names = ['bank', 'cifar10', 'cnae9', 'coil20', 'epileptic', 'fashion_mnist', 'fmd', 'har', 'hatespeech', 
                 'hiva', 'imdb', 'secom', 'seismic', 'sentiment', 'sms', 'spambase', 'svhn']
datasets = wgmm.load_datasets(dataset_names)

# Get CSV file ready
csv_file = "./results/measures.csv"
os.makedirs(os.path.dirname(csv_file), exist_ok=True)
if os.path.exists(csv_file):
    os.remove(csv_file)

results = []

for dataset_name, dataset in datasets.items():
    X, y = dataset

    # Fit GMMs to each label for this dataset
    gmms = wgmm.fit_gmms(dataset_name, dataset)

    # Calculate projection matrix with equal weights
    label_weights = y["Label"].value_counts(normalize=True).to_dict()
    weights = np.array([label_weights[label] for label in gmms.keys()])
    P = wgmm.calculate_projmat(gmms, weights, n_dims=2)

    # Grid for density estimation
    xx, yy = wgmm.calculate_grid(gmms, P)
    xx, yy = np.meshgrid(xx, yy)
    grid = (xx, yy)

    projections = {
        "WGMM-UAPCA": wgmm.wgmm_uapca(gmms, P),
        "UAPCA": wgmm.uapca(gmms, P),
        "PCA": wgmm.pca(dataset, P)
    }

    # Calculate densities for each projection method
    densities = {method: wgmm.calculate_density(proj, grid) for method, proj in projections.items()}

    # Project original data and plot each projection
    X_proj = X.values @ P
    wgmm.plot_and_save_projections(dataset_name, X_proj, y, densities, grid)

    comparisons = [
        ("PCA", "WGMM-UAPCA"),
        ("PCA", "UAPCA")
    ]

    # Compute measures for each comparison
    for ref_name, comp_name in comparisons:
        kl_sum = 0
        swd_sum = 0
        for label in gmms.keys():
            ref_density = densities[ref_name][label]
            comp_density = densities[comp_name][label]

            # KL Divergence
            kl_sum += label_weights[label] * compute_kl_divergence(ref_density, comp_density, grid)

            # Sliced Wasserstein Distance
            swd_sum += label_weights[label] * compute_sliced_wasserstein_distance(ref_density, comp_density, n_samples=5000, n_projections=50)

        results.append({
            "dataset_name": dataset_name,
            "ref_approach": ref_name,
            "comp_approach": comp_name,
            "kl_divergence": kl_sum,
            "wasserstein_distance": swd_sum
        })

# Save results to CSV
measures_df = pd.DataFrame(results)
measures_df.to_csv(csv_file, index=False)
print(f"Evaluation Successful. Results have been saved to: {csv_file}")